# Background

In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need! You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this brief tutorial to understand quickly how to use it.

In [150]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

# 1. Obtain the 200 top-ranking universities in www.topuniversities.com

In [151]:
URL = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508168782318'
req = requests.get(URL)

In [314]:
import json
from pandas.io.json import json_normalize

data = json.loads(req.text)
data = json_normalize(data['data'])[:200]

df_top = pd.DataFrame(data)

df_top = df_top[['rank_display','title','country','region','score','stars']]
df_top.head()

200

In [398]:
df_top['total_faculties'] = 'NaN'
df_top['inter_faculties'] = 'NaN'
df_top['total_students'] = 'NaN'
df_top['inter_students'] = 'NaN'

for i in range(200):
    r =  requests.get('https://www.topuniversities.com'+data.loc[i].url)
    soup = BeautifulSoup(r.text,'html.parser')
    numbers = soup.find_all('div','number')
    
    #Handling missing data
    if(len(numbers)!=8):
        print('Missing data for '+df_top['title'].loc[i]+' in row '+str(i))
        
    else:
        df_top['total_faculties'].loc[i] = float(numbers[0].text.replace(',', ''))
        df_top['inter_faculties'].loc[i]=float(numbers[1].text.replace(',', ''))
        df_top['total_students'].loc[i]= float(numbers[2].text.replace(',', ''))
        df_top['inter_students'].loc[i] = float(numbers[3].text.replace(',', ''))





Missing data for New York University (NYU) in row 51
Missing data for Indian Institute of Science (IISc) Bangalore in row 189


In [373]:
#Handling missing data
df_top['total_faculties'].loc[189] = 423
df_top['inter_faculties'].loc[189]='Nan'
df_top['total_students'].loc[189]= 4071
df_top['inter_students'].loc[189] = 47

df_top.head()

rank_display                                         title         country  \
0            1   Massachusetts Institute of Technology (MIT)   United States   
1            2                           Stanford University   United States   
2            3                            Harvard University   United States   
3            4  California Institute of Technology (Caltech)   United States   
4            5                       University of Cambridge  United Kingdom   

          region score stars total_faculties inter_faculties total_students  \
0  North America   100     6            2982            1679          11067   
1  North America  98.7     5            4285            2042          15878   
2  North America  98.4     5            4350            1311          22429   
3  North America  97.7     5             953             350           2255   
4         Europe  95.6     5            5490            2278          18770   

  inter_students  
0           3717  
1           3611  
2           5266  
3            647  
4           6699

# Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?

In [341]:
df_top_stat = df_top.copy()
df_top_stat= df_top_stat.drop(df_top_stat.index[51])

df_top_stat['ratio_faculty_student'] = df_top_stat.total_faculties / df_top_stat.total_students
df_top_stat['ratio_international_student'] = df_top_stat.inter_students / df_top_stat.total_students


Faculty members and students

In [372]:
df_top_stat.sort_values('ratio_faculty_student', ascending=False).head()

rank_display                                         title         country  \
3             4  California Institute of Technology (Caltech)   United States   
15           16                               Yale University   United States   
5             6                          University of Oxford  United Kingdom   
4             5                       University of Cambridge  United Kingdom   
16           17                      Johns Hopkins University   United States   

           region score stars total_faculties inter_faculties total_students  \
3   North America  97.7     5             953             350           2255   
15  North America  90.4     5            4940            1708          12402   
5          Europe  95.3     5            6750            2964          19720   
4          Europe  95.6     5            5490            2278          18770   
16  North America  89.8  None            4462            1061          16146   

   inter_students ratio_faculty_student ratio_international_student  
3             647              0.422616                    0.286918  
15           2469              0.398323                    0.199081  
5            7353              0.342292                     0.37287  
4            6699              0.292488                    0.356899  
16           4105              0.276353                    0.254243

International students

In [371]:
df_top_stat.sort_values('ratio_international_student', ascending=False).head()

rank_display                                              title  \
34            35  London School of Economics and Political Scien...   
11            12    Ecole Polytechnique Fédérale de Lausanne (EPFL)   
7              8                            Imperial College London   
198          200                              Maastricht University   
47           =47                         Carnegie Mellon University   

            country         region score stars total_faculties  \
34   United Kingdom         Europe  81.8  None            1088   
11      Switzerland         Europe  91.2  None            1695   
7    United Kingdom         Europe  93.7  None            3930   
198     Netherlands         Europe  47.9  None            1277   
47    United States  North America  78.6  None            1342   

    inter_faculties total_students inter_students ratio_faculty_student  \
34              687           9760           6748              0.111475   
11             1300          10343           5896              0.163879   
7              2071          16090           8746              0.244251   
198             502          16385           8234             0.0779371   
47              425          13356           6385              0.100479   

    ratio_international_student  
34                     0.691393  
11                     0.570047  
7                      0.543567  
198                    0.502533  
47                     0.478062

# Answer the previous question aggregating the data by (c) country and (d) region.

In [390]:
df_top_stat_agg = df_top_stat.copy()

Aggregate by country

In [399]:
df_top_stat_country = df_top_stat_agg[['country','total_faculties','inter_faculties','total_students','inter_students','ratio_faculty_student','ratio_international_student']]
df_top_stat_country = df_top_stat_country.groupby(by=['country']).agg(sum)

In [400]:
df_top_stat_country.sort_values('ratio_faculty_student', ascending=False).head()

total_faculties  total_students  inter_students  \
country                                                           
United States          152806.0       1264839.0        218877.0   
United Kingdom          79934.0        583621.0        199426.0   
Japan                   28395.0        186222.0         16269.0   
Germany                 33426.0        342499.0         56455.0   
Netherlands             20287.0        197631.0         46044.0   

                ratio_faculty_student  ratio_international_student  
country                                                             
United States                6.952958                     8.960914  
United Kingdom               3.805577                     9.836617  
Japan                        1.402556                     0.809999  
Germany                      1.083966                     1.818300  
Netherlands                  1.053113                     2.454562

In [401]:
df_top_stat_country.sort_values('ratio_international_student', ascending=False).head()

total_faculties  total_students  inter_students  \
country                                                           
United Kingdom          79934.0        583621.0        199426.0   
United States          152806.0       1264839.0        218877.0   
Australia               22034.0        301994.0        106359.0   
Netherlands             20287.0        197631.0         46044.0   
Switzerland             15323.0        109112.0         32995.0   

                ratio_faculty_student  ratio_international_student  
country                                                             
United Kingdom               3.805577                     9.836617  
United States                6.952958                     8.960914  
Australia                    0.687659                     3.121902  
Netherlands                  1.053113                     2.454562  
Switzerland                  0.944104                     2.196715

Aggregate by region

In [402]:
df_top_stat_region = df_top_stat_agg[['region','total_faculties','inter_faculties','total_students','inter_students','ratio_faculty_student','ratio_international_student']]
df_top_stat_region = df_top_stat_region.groupby(by=['region']).agg(sum)

In [403]:
df_top_stat_region.sort_values('ratio_faculty_student', ascending=False).head()

total_faculties  total_students  inter_students  \
region                                                           
Europe                218358.0       1957251.0        449364.0   
North America         182123.0       1546353.0        292116.0   
Asia                  106734.0        807003.0        110100.0   
Oceania                25347.0        350167.0        118798.0   
Latin America          45382.0        435750.0         36871.0   

               ratio_faculty_student  ratio_international_student  
region                                                             
Europe                     10.680289                    21.887973  
North America               7.676045                    10.729139  
Asia                        5.117563                     5.030969  
Oceania                     0.825033                     3.619843  
Latin America               0.677452                     0.502260

In [404]:
df_top_stat_region.sort_values('ratio_international_student', ascending=False).head()

total_faculties  total_students  inter_students  \
region                                                           
Europe                218358.0       1957251.0        449364.0   
North America         182123.0       1546353.0        292116.0   
Asia                  106734.0        807003.0        110100.0   
Oceania                25347.0        350167.0        118798.0   
Latin America          45382.0        435750.0         36871.0   

               ratio_faculty_student  ratio_international_student  
region                                                             
Europe                     10.680289                    21.887973  
North America               7.676045                    10.729139  
Asia                        5.117563                     5.030969  
Oceania                     0.825033                     3.619843  
Latin America               0.677452                     0.502260